# **3. 베이스라인 모델**

- 이제부터 파이토치를 활용해 딥러닝 모델을 만들자

- 파이토치를 활용한 딥러닝 모델링 절차는 다음과 같다:
    1. **시드값 고정 및 GPU 장비 설정**
    2. **데이터 준비**
        1. 훈련/검증 데이터 분리
        2. 데이터셋 클래스 정의
        3. 데이터셋 생성
        4. 데이터 로더 생성
            - 데이터셋으로부터 데이터를 배치 단위로 불러와주는 객체
    3. **모델 생성(CNN)**
    4. **모델 훈련**
        1. 손실 함수와 옵티마이저 설정
        2. 모델 훈련
    5. **성능 검증**
    6. **예측 및 제출**

# 3-1. 시드값 고정 및 GPU 장비 설정

## 3-1-1. 시드값 고정

In [2]:
import torch # 파이토치 
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)                # 파이썬 난수 생성기 시드 고정
np.random.seed(seed)             # 넘파이 난수 생성기 시드 고정
torch.manual_seed(seed)          # 파이토치 난수 생성기 시드 고정 (CPU 사용 시)
torch.cuda.manual_seed(seed)     # 파이토치 난수 생성기 시드 고정 (GPU 사용 시)
torch.cuda.manual_seed_all(seed) # 파이토치 난수 생성기 시드 고정 (멀티GPU 사용 시)
torch.backends.cudnn.deterministic = True # 확정적 연산 사용
torch.backends.cudnn.benchmark = False    # 벤치마크 기능 해제
torch.backends.cudnn.enabled = False      # cudnn 사용 해제

## 3-1-2. GPU 장비 설정
- 이어서 장비(device)를 설정해야 함
- 비정형 데이터(이미지, 음성, 텍스트 등)를 모델링하려면 연산량이 많아진다.
    - CPU로는 감당하기 벅찰 정도

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [6]:
device

device(type='cuda')

- CUDA는 엔비디아에서 개발한 병렬 처리 플랫폼

# 3-2. 데이터 준비

- 데이터 로더는 딥러닝 모델의 훈련에 필요한 데이터를 미니배치 단위로 공급하는 역할
- 이때, 데이터셋 클래스에 정의된 변환기가 원본 데이터를 다양한 형태로 변환해준다.

In [7]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [8]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압축 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

## 3-2-1. 훈련 데이터, 검증 데이터 분리

In [9]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=50)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

훈련 데이터 개수: 15750
검증 데이터 개수: 1750


## 3-2-2. 데이터셋 클래스 정의

- 사용자 정의 데이터셋을 만들자
- 파이토치로 신경망 모델을 구축하려면 데이터셋도 일정한 형식에 맞게 정의해야 함


</br>

- 파이토치에서 제공하는 `Dataset` 클래스를 활용해 데이터셋 객체를 만들 수 있다.
- `Dataset`은 추상 클래스
    - 추상 클래스는 곧바로 객체를 생성할 수 없고 상속만 할 수 있는 클래스를 일컫는다.
    - 추상 클래스를 사용하는 이유는 상속받는 클래스들의 메서드를 규격화하기 위해서이다.
    - 상속을 강제해 메서드 시그니처를 일치시킨다.
    
    
</br>

- 우리는 `Dataset`을 상속받은 다음, 특수 메서드인 `__len__()`과 `__getitem__()`을 재정의(오버라이딩)해야 함.
    - `__len__()`: 데이터셋 크기를 반환
    - `__getitem__()`: 인덱스를 전달받아 인덱스에 해당하는 데이터를 반환

In [11]:
import cv2 # OpenCV 라이브러리
from torch.utils.data import Dataset # 데이터 생성을 위한 클래스

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__() # 상속받은 Dataset의 생성자 호출
        # 전달받은 인수들 저장
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # 데이터셋 크기 반환 메서드 
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 반환 메서드 
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]    # 이미지 ID
        img_path = self.img_dir + img_id # 이미지 파일 경로 
        image = cv2.imread(img_path)     # 이미지 파일 읽기 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        label = self.df.iloc[idx, 1]     # 이미지 레이블(타깃값)

        if self.transform is not None:
            image = self.transform(image) # 변환기가 있다면 이미지 변환
        return image, label

## 3-2-3. 데이터셋 생성
- 앞서 정의한 `ImageDataset` 클래스를 이용하여 데이터셋을 만들어보자.
- 파이토치 모델로 이미지를 다루려면 이미지 데이터를 **텐서(tensor)** 타입으로 바꿔야 한다.

In [12]:
from torchvision import transforms # 이미지 변환을 위한 모듈

transform = transforms.ToTensor()

- `ToTensor()` 메서드로 이미지를 텐서로 바꿨다.
    - (가로 픽셀 수, 세로 픽셀 수, 채널수) $\Rightarrow$ (채널수, 가로 픽셀 수, 세로 픽셀 수)
    - $32 \times 32 \times 3$ $\Rightarrow$ $3 \times 32 \times 32$
    
    
- 이제, 훈련 데이터셋과 검증 데이터셋을 만들자.
    - 앞서 정의한 `ImageDataset()`클래스를 사용하면 된다.

In [13]:
dataset_train = ImageDataset(df=train, img_dir='train/', transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform)

## 3-2-4. 데이터 로더 생성

- 데이터셋을 생성했다.
- 그 다음으로는 지정한 배치 크기만큼씩 데이터를 불러오는 객체인 데이터 로더를 생성해야 함.
    - 딥러닝 모델을 훈련할 때는 주로 배치 단위로 데이터를 가져와 훈련함.
    - 배치의 크기는 2의 제곱수로 설정하는 것이 효율적

In [14]:
from torch.utils.data import DataLoader # 데이터 로더 클래스

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

# 3-3. 모델 생성

In [15]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

- CNN 모델은 `nn.Module`을 상속해 정의함
- 그리고 순전파 후 결과를 반환하는 메서드인 `forward()`를 재정의
- 참고로, 파이토치에서 `nn.Module`은 모든 신경망 모듈의 기반 클래스

In [16]:
class Model(nn.Module):
    # 신경망 계층 정의 
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출
        
        # 첫 번째 합성곱 계층 
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                               kernel_size=3, padding=2) 
        # 두 번째 합성곱 계층 
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, 
                               kernel_size=3, padding=2) 
        # 최대 풀링 계층 
        self.max_pool = nn.MaxPool2d(kernel_size=2) 
        # 평균 풀링 계층 
        self.avg_pool = nn.AvgPool2d(kernel_size=2) 
        # 전결합 계층 
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)
        
    # 순전파 출력 정의 
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x

- 마지막으로, 이렇게 정의한 `Model` 클래스로 CNN 모델을 생성하여 device 장비에 할당하자
- 현재 device는 GPU를 사용하도록 설정되어있다.

In [17]:
model = Model().to(device)

model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

# 3-4. 모델 훈련

- 모델을 훈련하기 전, 손실 함수와 옵티마이저를 정의

## 3-4-1. 손실 함수 설정

- 가중치 갱신은 예측값과 실젯값의 손실이 작아지는 방향으로
- 이때, 손실값을 구하는 함수가 손실 함수
- 여기서는 손실 함수로 교차 엔트로피를 사용

In [18]:
# 손실함수
criterion = nn.CrossEntropyLoss()

## 3-4-2. 옵티마이저 설정

- 옵티마이저는 최적 가중치를 찾아주는 알고리즘
- 기본 옵티마이저인 SGD로 설정

In [19]:
# 옵티마이저
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## 3-4-3. 모델 훈련

- 이미지용 딥러닝 모델 훈련 절차는 다음과 같다.

1. 데이터 로더에서 배치 크기만큼 데이터를 불러온다.
2. 불러온 이미지 데이터와 레이블(타깃값) 데이터를 장비(GPU 혹은 CPU)에 할당한다.
3. 옵티마이저 내 기울기를 초기화한다.
4. 신경망 모델에 입력 데이터(이미지)를 전달해 순전파하여 출력값(예측값)을 구한다.
5. 예측값과 실제 레이블(타깃값)을 비교해 손실을 계산한다.
6. 손실을 기반으로 역전파를 수행한다.
7. 역전파로 구한 기울기를 활용해 가중치를 갱신한다.
8. 1~7 절차를 반복 횟수만큼 되풀이한다.
9. 1~8 절차를 에폭만큼 반복한다.

In [20]:
epochs = 10 # 총 에폭
# 총 에폭만큼 반복
for epoch in range(epochs):
    epoch_loss = 0 # 에폭별 손실값 초기화
    
    # '반복 횟수'만큼 반복 
    for images, labels in loader_train:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device)
        
        # 옵티마이저 내 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item() 
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()
        
    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')

에폭 [1/10] - 손실값: 0.5233
에폭 [2/10] - 손실값: 0.3453
에폭 [3/10] - 손실값: 0.2370
에폭 [4/10] - 손실값: 0.1984
에폭 [5/10] - 손실값: 0.1754
에폭 [6/10] - 손실값: 0.1666
에폭 [7/10] - 손실값: 0.1545
에폭 [8/10] - 손실값: 0.1439
에폭 [9/10] - 손실값: 0.1333
에폭 [10/10] - 손실값: 0.1299


# 3-5. 성능 검증

In [21]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수 임포트

# 실제값과 예측 확률값을 담을 리스트 초기화
true_list = []
preds_list = []

In [22]:
model.eval() # 모델을 평가 상태로 설정 

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당 
        images = images.to(device)
        labels = labels.to(device) 
        
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률  
        true = labels.cpu() # 실제값 
        # 예측 확률과 실제값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)
        
# 검증 데이터 ROC AUC 점수 계산
print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.3f}')

검증 데이터 ROC AUC : 0.990


# 3-6. 예측 및 결과 제출

In [23]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

## 3-6-1. 예측

In [24]:
model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측값 저장용 리스트 초기화

with torch.no_grad(): # 기울기 계산 비활성화
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)
        
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃값이 1일 확률(예측값)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # preds에 preds_part 이어붙이기
        preds.extend(preds_part)

## 3-6-2. 결과 제출

In [25]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

- 이제 제출 전에 훈련 이미지 데이터와 테스트 이미지 데이터를 모두 삭제

In [26]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')